# Generate Images from Records by plotting and saving Matplotlib figures 

Import modules

In [ ]:
%matplotlib 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import wfdb
import os
from wfdb import processing
import numpy as np
import cv2

Download Database 

Plot all Signals

Record 100 is imported and ploted using 'wfdb' tools.

In [ ]:
record = wfdb.rdrecord('../dataset/mitdb/208', channels=[0])
annotation = wfdb.rdann('../dataset/mitdb/208','atr')
signal, fields = wfdb.rdsamp('../dataset/mitdb/208', channels=[0])

In [ ]:
wfdb.plot_wfdb(record=record, annotation=annotation, plot_sym=True, figsize=(20,8))

In [ ]:
figure = []

record = wfdb.rdrecord('../dataset/mitdb/100', sampfrom=370-200, sampto=370+200, channels=[0])
figure.append(wfdb.plot_wfdb(record=record, title=370, return_fig=True))

record = wfdb.rdrecord('../dataset/mitdb/100', sampfrom=370, sampto=370+200, channels=[0])
figure.append(wfdb.plot_wfdb(record=record, title=370, return_fig=True))

figure[0].savefig('../screenshots/report_pics/description_of_the_process/' + "/" + 'full_plot' + ".png")
figure[1].savefig('../screenshots/report_pics/description_of_the_process/' + "/" + 'rt_plot' + ".png")

image = figure[1]

image.canvas.draw()

data = np.frombuffer(image.canvas.tostring_rgb(), dtype=np.uint8)

data = data.reshape((image.canvas.get_width_height()[::-1] + (3,)))

data = cv2.cvtColor(data, cv2.COLOR_RGB2BGR)

data = data[288-250:288-35, 55:390]

cv2.imwrite('../screenshots/report_pics/description_of_the_process/' + "/" + 'cropped_rt_plot' + ".png", data)

The data structure used to store the signal is Numpy Array. 'p_signal' attribute consists the signal data.
The Header file consists of information about the person from whom the signal is extracted.  

In [ ]:
# 'sample' and 'symbol'
annotation.__dict__

In [ ]:
record.__dict__

In [ ]:
signal[:,0]

In [ ]:
fields

QRS segment detection using 'wfdb'. 

In [ ]:
rqrs = processing.xqrs_detect(record.p_signal[:,0], record.fs)

In [ ]:
len(rqrs)

The record number is stored in 'data'. R-T intervals are plotted and stored in 'figure'.

100 - normal

118 - rbbb

217 - paced

106, 208, 233 - PVC

In [ ]:
recs = ["100", "118", "217"]
recs = ["106"]

In [ ]:
dataset = {}
for rec_no in recs:
    data = "../dataset/mitdb/"+rec_no
    result_to = "../dataset/images/"+rec_no
    record = wfdb.rdrecord(data, channels=[0])
    rqrs = processing.xqrs_detect(record.p_signal[:,0], record.fs)
    figure = []
    for i in range(100):
        start = rqrs[i]-200
        stop = rqrs[i]+200
        if start<0: 
            start=0
        temp_rec = wfdb.rdrecord(data, sampfrom=start,sampto=stop, channels=[0])
        figure.append(wfdb.plot_wfdb(record=temp_rec, annotation=temp_ann, plot_sym=True, title=rqrs[i], return_fig=True))
        figure[-1].savefig(result_to + "/" + str(rqrs[i]) + ".png")
        plt.close()
    dataset[rec_no] = figure

# Convert Matplotlib Figures to Numpy arrays

In [ ]:
image = dataset["100"][0]

In [ ]:
image.canvas.draw()

In [ ]:
data = np.frombuffer(image.canvas.tostring_rgb(), dtype=np.uint8)

In [ ]:
data = data.reshape((image.canvas.get_width_height()[::-1] + (3,)))

In [ ]:
data.shape

In [ ]:
data = cv2.cvtColor(data, cv2.COLOR_RGB2BGR)

In [ ]:
data = data[288-250:288-35, 55:390]

In [ ]:
data = cv2.resize(data, (int(215/5), int(215/5)), cv2.INTER_LINEAR)

In [ ]:
cv2.imshow('image', data)

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

# New Image Generation 

106 - PVC <br>
109 - LBBB <br>
213 - Fusion of PVC and Normal

In [ ]:
record = wfdb.rdrecord('../dataset/mitdb/111', channels=[0])
annotation = wfdb.rdann('../dataset/mitdb/111','atr')

In [ ]:
wfdb.plot_wfdb(record=record, annotation=annotation, plot_sym=True)

In [ ]:
recs = ['109']
label = 'L'

In [ ]:
dataset = {}
for rec_no in recs:
    data = "../dataset/mitdb/"+rec_no
    result_to = "../dataset/images-half/"+rec_no
    figure = []
    for i in range(115):
        if annotation.symbol[i] == label:
            start = annotation.sample[i]
            stop = annotation.sample[i]+200
            if start<0: 
                start=0
            temp_rec = wfdb.rdrecord(data, sampfrom=start,sampto=stop, channels=[0])
            figure.append(wfdb.plot_wfdb(record=temp_rec, plot_sym=True, title=annotation.sample[i], return_fig=True))
            figure[-1].savefig(result_to + "/" + str(annotation.sample[i]) + ".png")
            plt.show()
            plt.close()
    dataset[rec_no] = figure